In [123]:
import os
import json
import numpy as np
import pandas as pd
import sqlite3
import functools as ft
import matplotlib.pyplot as plt
%matplotlib inline

ETL - Extract, Transform, Load

Extract: Our data is extracted from an Excel file named 'Main tests results table.xlsx' . In the ETL process, the data is first extracted so that we can work on it.

In [124]:
Main_tests_results_df = pd.read_excel('Main tests results table.xlsx')
Main_tests_results_df

,Date,Patient Number,Routine test,S/P MI,S/P CABG,Heart failure,EF%,HTN,DM,Dyslipidemia,...,Aldactone/Spironolactone,Eliquis,Dose,Xarelto,Dose.1,Pradaxa,Dose.2,Coumadin,Dose.3,Fried Scale
0,2005-06-30,1,1,0.0,0.0,0,20.0,1.0,1.0,1.0,...,0.0,1.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,4.0
1,2005-07-08,1,1,1.0,0.0,1,40.0,1.0,0.0,1.0,...,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,3.0
2,2005-10-24,1,1,0.0,0.0,1,60.0,1.0,0.0,1.0,...,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,5.0
3,2006-01-08,1,1,1.0,0.0,1,25.0,1.0,0.0,1.0,...,0.0,1.0,NaN,0.0,20.0,0.0,NaN,0.0,NaN,4.0
4,2006-02-02,1,1,1.0,0.0,0,60.0,1.0,0.0,0.0,...,0.0,1.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2019-10-09,300,0,0.0,0.0,0,60.0,1.0,1.0,1.0,...,0.0,1.0,2.5,0.0,NaN,0.0,NaN,0.0,NaN,2.0
99996,2019-11-19,300,0,1.0,0.0,1,60.0,1.0,0.0,0.0,...,0.0,1.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,2.0
99997,2019-11-21,300,0,0.0,1.0,1,60.0,1.0,0.0,1.0,...,0.0,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,1.0
99998,2019-12-17,300,0,0.0,0.0,1,65.0,1.0,0.0,1.0,...,0.0,1.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,4.0


In [125]:
Main_tests_results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 49 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   Date                      100000 non-null  datetime64[ns]
 1   Patient Number            100000 non-null  int64         
 2   Routine test              100000 non-null  int64         
 3   S/P MI                    98666 non-null   float64       
 4   S/P CABG                  98682 non-null   float64       
 5   Heart failure             98764 non-null   object        
 6   EF%                       98148 non-null   float64       
 7   HTN                       98724 non-null   float64       
 8   DM                        98626 non-null   float64       
 9   Dyslipidemia              98742 non-null   float64       
 10  CVA/TIA                   98674 non-null   float64       
 11  PVD                       98648 non-null   float64       
 12  Mod

In [126]:
Main_tests_results_df.shape

(100000, 49)

Transform: The ETL's central process is in which various changes are made to the original data so that we can adapt the data to the project's goals. In addition, in the process, we clean the data of empty values ​​and noise so that they do not interfere with drawing conclusions from the data.

In [127]:
Main_tests_results_df.isnull().sum()

Date                            0
Patient Number                  0
Routine test                    0
S/P MI                       1334
S/P CABG                     1318
Heart failure                1236
EF%                          1852
HTN                          1276
DM                           1374
Dyslipidemia                 1258
CVA/TIA                      1326
PVD                          1352
Moderate-Severe AS           1328
Moderate-Severe AR           1222
Moderate-Severe MS           1320
Moderate-Severe MR           1378
Atrial fibrillation          1384
Pacemaker                    1270
ICD                          1376
CRTD                         1328
Creatinine                   1230
HgB                          1260
Albumin                      1272
LDL                          1350
HDL                          1706
TG                           1658
NT-proBNP                   97624
Aspirin                      1322
Plavix                       1342
Brilinta      

Because there are columns that have a large number of empty values/contain only empty values ​​we get an empty table.
Therefore we will limit throwing empty values ​​to specific columns that have 50% of their values ​​empty.

Selecting and dropping blank values ​​from certain columns

In [128]:
res = Main_tests_results_df.isnull().sum()
res[res > 25000]

NT-proBNP    97624
Dose         69272
Dose.1       96014
Dose.2       97550
Dose.3       97848
dtype: int64

Clear all 50% Null value from the columns

In [129]:
Main_tests_results_df.drop(['NT-proBNP', 'Dose','Dose.1', 'Dose.2', 'Dose.3'],axis='columns', inplace=True)
Main_tests_results_df.head()

,Date,Patient Number,Routine test,S/P MI,S/P CABG,Heart failure,EF%,HTN,DM,Dyslipidemia,...,Statins.1,Entresto,Jardiance/forxiga,Fusid,Aldactone/Spironolactone,Eliquis,Xarelto,Pradaxa,Coumadin,Fried Scale
0,2005-06-30,1,1,0.0,0.0,0,20.0,1.0,1.0,1.0,...,LIPITOR 40,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0
1,2005-07-08,1,1,1.0,0.0,1,40.0,1.0,0.0,1.0,...,atozit 10/20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
2,2005-10-24,1,1,0.0,0.0,1,60.0,1.0,0.0,1.0,...,ezetrol 10,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,2006-01-08,1,1,1.0,0.0,1,25.0,1.0,0.0,1.0,...,SIMOVIL 20,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0
4,2006-02-02,1,1,1.0,0.0,0,60.0,1.0,0.0,0.0,...,lipitor 80,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


Now we can discard all other empty values

In [130]:
Main_tests_results_df.isnull().sum()

Date                            0
Patient Number                  0
Routine test                    0
S/P MI                       1334
S/P CABG                     1318
Heart failure                1236
EF%                          1852
HTN                          1276
DM                           1374
Dyslipidemia                 1258
CVA/TIA                      1326
PVD                          1352
Moderate-Severe AS           1328
Moderate-Severe AR           1222
Moderate-Severe MS           1320
Moderate-Severe MR           1378
Atrial fibrillation          1384
Pacemaker                    1270
ICD                          1376
CRTD                         1328
Creatinine                   1230
HgB                          1260
Albumin                      1272
LDL                          1350
HDL                          1706
TG                           1658
Aspirin                      1322
Plavix                       1342
Brilinta                     1370
Effient       

In [131]:
Main_tests_results_df = Main_tests_results_df.dropna()
Main_tests_results_df

,Date,Patient Number,Routine test,S/P MI,S/P CABG,Heart failure,EF%,HTN,DM,Dyslipidemia,...,Statins.1,Entresto,Jardiance/forxiga,Fusid,Aldactone/Spironolactone,Eliquis,Xarelto,Pradaxa,Coumadin,Fried Scale
0,2005-06-30,1,1,0.0,0.0,0,20.0,1.0,1.0,1.0,...,LIPITOR 40,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0
3,2006-01-08,1,1,1.0,0.0,1,25.0,1.0,0.0,1.0,...,SIMOVIL 20,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0
6,2006-04-08,1,1,1.0,1.0,1,60.0,1.0,0.0,1.0,...,LIPITOR 20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2007-02-02,1,1,0.0,0.0,1,15.0,1.0,1.0,1.0,...,lipitor 40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
10,2007-04-27,1,1,0.0,0.0,1,55.0,1.0,1.0,1.0,...,ATOZET 10/20,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,2019-10-09,300,0,0.0,0.0,אין אקו,20.0,1.0,1.0,1.0,...,LITORVA 10,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
99995,2019-10-09,300,0,0.0,0.0,0,60.0,1.0,1.0,1.0,...,SIMVACOR 20,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
99996,2019-11-19,300,0,1.0,0.0,1,60.0,1.0,0.0,0.0,...,atozit 10/20,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
99998,2019-12-17,300,0,0.0,0.0,1,65.0,1.0,0.0,1.0,...,LIPITOR 40,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0


Erasing errors in the data

In [132]:
Main_tests_results_df['Heart failure'].value_counts()

Heart failure
1           30372
0           15784
אין אקו       328
Name: count, dtype: int64

In [133]:
errors = ['Heart failure', 'Creatinine', 'HgB', 'Albumin', 'LDL', 'HDL', 'TG']

for des in errors:
    ero = Main_tests_results_df[des].str.extract(pat='(\w+)', expand=False)
    Main_tests_results_df.drop(ero[~ero.isna()].index, inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_55248\4225312352.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_tests_results_df.drop(ero[~ero.isna()].index, inplace=True)


In [134]:
Main_tests_results_df['Heart failure'].value_counts()

Heart failure
1    28986
0    15116
Name: count, dtype: int64

Converts all values ​​in the categorical column to uppercase

In [135]:
Main_tests_results_df['Statins.1'] = Main_tests_results_df['Statins.1'].apply(lambda x: ''.join([i.upper() if i.isalpha() else i for i in str(x)]))
Main_tests_results_df['Statins.1'].value_counts()

C:\Users\user\AppData\Local\Temp\ipykernel_55248\873072643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_tests_results_df['Statins.1'] = Main_tests_results_df['Statins.1'].apply(lambda x: ''.join([i.upper() if i.isalpha() else i for i in str(x)]))


Statins.1
LIPITOR 40      6762
LIPITOR 20      5766
LIPITOR 80      2320
LIPITOR 10      1832
LIPITOR 40      1548
                ... 
SIMACOR 40       164
SIMVAXON 20      160
ATOZET10/10      160
ATOZIT 10/80     156
ATOZET10/20      142
Name: count, Length: 67, dtype: int64

We will check if we have duplicate values ​​in the data

In [136]:
Main_tests_results_df.duplicated().sum()

0

Sort the DF by Patient Number and then by date.
In addition, we calculated the differences between any two consecutive tests.

In [137]:
Main_tests_results_df.sort_values(by=['Patient Number', 'Date'], inplace=True)
Main_tests_results_df

C:\Users\user\AppData\Local\Temp\ipykernel_55248\3205238161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Main_tests_results_df.sort_values(by=['Patient Number', 'Date'], inplace=True)


,Date,Patient Number,Routine test,S/P MI,S/P CABG,Heart failure,EF%,HTN,DM,Dyslipidemia,...,Statins.1,Entresto,Jardiance/forxiga,Fusid,Aldactone/Spironolactone,Eliquis,Xarelto,Pradaxa,Coumadin,Fried Scale
0,2005-06-30,1,1,0.0,0.0,0,20.0,1.0,1.0,1.0,...,LIPITOR 40,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0
50000,2005-06-30,1,0,0.0,0.0,0,20.0,1.0,1.0,1.0,...,LIPITOR 40,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0
6,2006-04-08,1,1,1.0,1.0,1,60.0,1.0,0.0,1.0,...,LIPITOR 20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50006,2006-04-08,1,0,1.0,1.0,1,60.0,1.0,0.0,1.0,...,LIPITOR 20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2007-02-02,1,1,0.0,0.0,1,15.0,1.0,1.0,1.0,...,LIPITOR 40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,2019-11-19,300,0,1.0,0.0,1,60.0,1.0,0.0,0.0,...,ATOZIT 10/20,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
49998,2019-12-17,300,1,0.0,0.0,1,65.0,1.0,0.0,1.0,...,LIPITOR 40,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0
99998,2019-12-17,300,0,0.0,0.0,1,65.0,1.0,0.0,1.0,...,LIPITOR 40,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0
49999,2019-12-26,300,1,1.0,1.0,0,40.0,1.0,1.0,1.0,...,LIPITOR 10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


Load: Loading all the tables and merging them into one final table. Because in our project we worked only on a Fact table, we will present the final table after the Transform.

In [138]:
final_Main_tests_results_df = Main_tests_results_df.copy()
final_Main_tests_results_df

,Date,Patient Number,Routine test,S/P MI,S/P CABG,Heart failure,EF%,HTN,DM,Dyslipidemia,...,Statins.1,Entresto,Jardiance/forxiga,Fusid,Aldactone/Spironolactone,Eliquis,Xarelto,Pradaxa,Coumadin,Fried Scale
0,2005-06-30,1,1,0.0,0.0,0,20.0,1.0,1.0,1.0,...,LIPITOR 40,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0
50000,2005-06-30,1,0,0.0,0.0,0,20.0,1.0,1.0,1.0,...,LIPITOR 40,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.0
6,2006-04-08,1,1,1.0,1.0,1,60.0,1.0,0.0,1.0,...,LIPITOR 20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
50006,2006-04-08,1,0,1.0,1.0,1,60.0,1.0,0.0,1.0,...,LIPITOR 20,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9,2007-02-02,1,1,0.0,0.0,1,15.0,1.0,1.0,1.0,...,LIPITOR 40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,2019-11-19,300,0,1.0,0.0,1,60.0,1.0,0.0,0.0,...,ATOZIT 10/20,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
49998,2019-12-17,300,1,0.0,0.0,1,65.0,1.0,0.0,1.0,...,LIPITOR 40,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0
99998,2019-12-17,300,0,0.0,0.0,1,65.0,1.0,0.0,1.0,...,LIPITOR 40,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,4.0
49999,2019-12-26,300,1,1.0,1.0,0,40.0,1.0,1.0,1.0,...,LIPITOR 10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
